In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM

No dicts found; please check load_dicts...
Optimize a model with 8432 rows, 8432 columns and 8432 nonzeros
Model has 35553528 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e+00, 2e+07]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 8432 rows and 0 columns (presolve time = 9s) ...
Presolve removed 8432 rows and 0 columns
Presolve time: 8.91s
Presolved: 0 rows, 8432 columns, 0 nonzeros
Presolved model has 35553528 quadratic objective terms
Ordering time: 0.20s

Barrier statistics:
 Free vars  : 5661
 AA' NZ     : 1.602e+07
 Factor NZ  : 1.603e+07 (roughly 140 MBytes of memory)
 Factor Ops : 6.049e+10 (roughly 6 seconds per iteration)
 Threads    : 12

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.43919149e+11 -2.22738455e+05  5.71e+05 5.07e+03  9.94e+05    44s
   1  -1.10454253e+10 -1.29162791e+08  4.44e+04 3

In [2]:
# x = GLS_Apr_weekday_PM.x; # sample matrix, each column is a link flow vector sample; number_of_links * K
# S = GLS_Apr_weekday_PM.S; # sample covariance matrix
# A = GLS_Apr_weekday_PM.A; # link_route incidence matrix
P = GLS_Apr_weekday_PM.P; # route_choice_probability_matrix
# Q = GLS_Apr_weekday_PM.Q; 
L = GLS_Apr_weekday_PM.L; # dimension of xi
number_of_routes = GLS_Apr_weekday_PM.number_of_routes;
number_of_links = GLS_Apr_weekday_PM.number_of_links;

In [3]:
# A = sparse(A);
P = sparse(P);
# Q = sparse(Q);

In [4]:
# K = size(x, 2)

# inv_S = inv(S)

# A_t = transpose(A)

# b = sum([A_t * inv_S * x[:, k] for k = 1:K])

In [5]:
using JuMP

### implement GLS method to estimate OD demand matrix

In [6]:
# model = Model(solver=IpoptSolver())

# @variable(model, xi[1:L] >= 0)

# # Set objective: (K/2) xi' * Q * xi - b' * xi
# obj = 0
# for i = 1:L
#     for j = 1:L
#         obj += (1.0 / 2) * K * xi[i] * Q[i, j] * xi[j]
#     end
# end
# for l = 1:L
#     obj += - b[l] * xi[l]
# end

# @NLobjective(model, Min, obj) 

# solve(model)

In [7]:
# xi_list = getvalue(xi)

xi_list = GLS_Apr_weekday_PM.xi_list

8432-element Array{Any,1}:
 319.643     
 233.954     
   0.0120999 
  70.6385    
 746.269     
   0.00862451
 288.642     
   0.0363114 
 266.362     
 395.605     
 309.833     
   0.0125291 
 393.056     
   ⋮         
   3.77333e-6
   9.96508e-6
   6.54659e-6
   4.92868e-6
   3.33238e-6
   8.97179e-6
   6.50675e-6
   4.09649e-6
   1.53308e-5
   0.0243625 
   0.00511183
   0.00551115

In [8]:
mGLSJulia = Model()

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...: 37834384
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............: 28382112

Total number of variables............................:  9461826
                     variables with only lower bounds:  9461826
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Error

In [9]:
getvalue(lam)

1122-element Array{Float64,1}:
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 ⋮         
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999
 0.00999999

In [10]:
getobjectivevalue(mGLSJulia)

84.31991567998695